### Process Overview:
- Input: sampled_100.json 

- Output: preprocessed_100.json

##### Text Preprocessing Steps: 
- Lowercasing: Convert all text to lowercase to ensure consitency 

- Tokenization: Split the text into individual words or tokens.  

- Removing Punctuation: Eliminate punctuation marks  

- Removing Stopwords: Remove common words like "the", "is"   

- Stemming or Lemmatization: Reduce words to  theirs root form  

- Removing Numbers and Special Characters  

- Removing Extra Whitespaces 

In [1]:
import json
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

def preprocess_corpus(input_file_path, output_file_path):
    # Open and read the JSON file
    with open(input_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Initialize the stopwords, stemmer, and other required objects
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Add custom stopwords
    custom_stopwords = {'said', 'could', 'might', 'like', 'also', 'would'}
    stop_words.update(custom_stopwords)

    # Define the preprocessing function:
    def preprocess_text(text):
        # Lowercasing
        text = text.lower() 

        # Tokenization
        tokens = word_tokenize(text) 

        # Removing Punctuation
        tokens = [word for word in tokens if word not in string.punctuation]

        # Removing Stopwords and single characters
        tokens = [word for word in tokens if word not in stop_words and len(word) > 1]

        # Lemmatization
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Removing Numbers and Special Characters
        tokens = [word for word in tokens if word.isalpha()]

        # Join tokens back into text
        processed_text = ' '.join(tokens)

        return processed_text

    # Preprocess each story in the data, retaining only the preprocessed text
    preprocessed_data = []
    for story in data:
        preprocessed_text = preprocess_text(story['story_text'])
        preprocessed_data.append(preprocessed_text)

    # Write the preprocessed data to the output file
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(preprocessed_data, f, ensure_ascii=False, indent=4)

# Example usage
input_file_path = 'sampled_300.json'
output_file_path = 'tokenized_300.json'
preprocess_corpus(input_file_path, output_file_path)
